In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("DateTime").getOrCreate()

24/04/29 15:30:55 WARN Utils: Your hostname, myspark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/04/29 15:30:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/29 15:31:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("/home/vboxuser/jupyter_notebooks/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/appl_stock.csv",
                    inferSchema = True,
                    header = True)

In [4]:
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [5]:
from pyspark.sql.functions import year, dayofmonth, dayofyear, weekofyear, month, hour, minute

In [6]:
df.select(year(df["Date"])).show()

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 20 rows



In [7]:
df.select(month(df["Date"])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [8]:
df.select(hour(df["Date"])).show()

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [9]:
df.select(dayofmonth(df["Date"])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



## Finding the Average Closing Amount per year of the Apple Stock

**Steps:**
1. Extract Year from the Date field.
2. Get Average Closing Amount grouped by year.
3. Format Avg Closing Amount to 2 decimal places and order by year

In [22]:
df_with_year = df.withColumn("Year", year(df["Date"]))

In [28]:
df_with_year.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
|2010-01-11|212.

In [36]:
avg_close = df_with_year.groupBy(df_with_year["Year"]).mean("Close")

In [37]:
avg_close.show()

+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [38]:
from pyspark.sql.functions import format_number

In [43]:
avg_close.select("Year",format_number("avg(Close)",2).alias("Avg Closing")).orderBy("Year").show()

+----+-----------+
|Year|Avg Closing|
+----+-----------+
|2010|     259.84|
|2011|     364.00|
|2012|     576.05|
|2013|     472.63|
|2014|     295.40|
|2015|     120.04|
|2016|     104.60|
+----+-----------+



The above code uses the groupBy() and mean() methods to determine the average of Closing value of Apple Stocks.

The aggregation can also be done using the agg() method. The end result is exactly the same, the following is just another way of doing the same thing.

In [50]:
grouped_data = df_with_year.groupBy(df_with_year["Year"])

In [56]:
avgclose = grouped_data.agg({"Close" : "avg"})

In [57]:
avgclose.select("Year",format_number("avg(Close)",2).alias("Avg Closing")).orderBy("Year").show()

+----+-----------+
|Year|Avg Closing|
+----+-----------+
|2010|     259.84|
|2011|     364.00|
|2012|     576.05|
|2013|     472.63|
|2014|     295.40|
|2015|     120.04|
|2016|     104.60|
+----+-----------+



The ouput of both the approaches is matching.